In [1]:
!pip freeze | grep tensorflow

tensorflow==2.5.0
tensorflow-addons==0.13.0
tensorflow-estimator==2.5.0
tensorflow-io==0.19.0
tensorflow-io-gcs-filesystem==0.19.0


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
!pip install ipython-autotime

In [4]:
%load_ext autotime
%load_ext tensorboard

time: 9.42 ms (started: 2021-07-06 11:54:27 +02:00)


In [5]:
! pip install git+https://github.com/sanpigh/MRIsegmentation.git@cleanup --upgrade

  Cloning https://github.com/sanpigh/MRIsegmentation.git (to revision cleanup) to /tmp/pip-req-build-iw3_yhhe
  Running command git clone -q https://github.com/sanpigh/MRIsegmentation.git /tmp/pip-req-build-iw3_yhhe
  Running command git checkout -b cleanup --track origin/cleanup
  Switched to a new branch 'cleanup'
  Branch 'cleanup' set up to track remote branch 'cleanup' from 'origin'.
time: 3.08 s (started: 2021-07-06 11:54:43 +02:00)


In [8]:
# drive.mount('/content/drive')

time: 468 µs (started: 2021-07-06 12:14:33 +02:00)


In [16]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from skimage import io

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
import tensorflow.keras.backend as K

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.applications import VGG19

from warnings import filterwarnings
filterwarnings('ignore')

import random

import glob
from IPython.display import display

time: 1.14 s (started: 2021-07-06 12:20:19 +02:00)


In [24]:
!pwd

/home/alvarocg/code/sanpigh/MRIsegmentation/notebooks
time: 146 ms (started: 2021-07-06 12:23:47 +02:00)


In [22]:
DATA_PATH = "/home/alvarocg/code/sanpigh/MRIsegmentation/raw_data/kaggle_3m/"

time: 743 µs (started: 2021-07-06 12:23:06 +02:00)


In [23]:
data = pd.read_csv(DATA_PATH + 'data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Patient                    110 non-null    object 
 1   RNASeqCluster              92 non-null     float64
 2   MethylationCluster         109 non-null    float64
 3   miRNACluster               110 non-null    int64  
 4   CNCluster                  108 non-null    float64
 5   RPPACluster                98 non-null     float64
 6   OncosignCluster            105 non-null    float64
 7   COCCluster                 110 non-null    int64  
 8   histological_type          109 non-null    float64
 9   neoplasm_histologic_grade  109 non-null    float64
 10  tumor_tissue_site          109 non-null    float64
 11  laterality                 109 non-null    float64
 12  tumor_location             109 non-null    float64
 13  gender                     109 non-null    float64

In [26]:
!find ../raw_data/kaggle_3m -type f -print | grep mask | wc -l    # files with mask suffix
!find ../raw_data/kaggle_3m -type f -print | grep -v mask | wc -l # files without mask suffix

3929
3931
time: 257 ms (started: 2021-07-06 12:24:48 +02:00)


In [96]:
# Raw data
# data_map = []
# for sub_dir_path in glob.glob(DATA_PATH+"*"):
#     if os.path.isdir(sub_dir_path):
#         dirname = sub_dir_path.split("/")[-1]
#         for filename in os.listdir(sub_dir_path):
#             image_path = sub_dir_path + "/" + filename
#             data_map.extend([dirname, image_path])
#     else:
#         print("This is not a dir:", sub_dir_path)

# Raw data
data_map = []
for sub_dir_path in glob.glob(DATA_PATH+"*"):
    #if os.path.isdir(sub_path_dir):
    try:
        dir_name = sub_dir_path.split('/')[-1]
        for filename in os.listdir(sub_dir_path):
            image_path = sub_dir_path + '/' + filename
            data_map.extend([dir_name, image_path])
    except Exception as e:
        print(e)

[Errno 20] Not a directory: '/home/alvarocg/code/sanpigh/MRIsegmentation/raw_data/kaggle_3m/data.csv'
[Errno 20] Not a directory: '/home/alvarocg/code/sanpigh/MRIsegmentation/raw_data/kaggle_3m/README.md'
time: 56.4 ms (started: 2021-07-06 15:41:50 +02:00)


In [97]:
df = pd.DataFrame({"patient_id" : data_map[::2],
                   "path" : data_map[1::2]})
df.head()

,patient_id,path
0,TCGA_DU_7294_19890104,/home/alvarocg/code/sanpigh/MRIsegmentation/ra...
1,TCGA_DU_7294_19890104,/home/alvarocg/code/sanpigh/MRIsegmentation/ra...
2,TCGA_DU_7294_19890104,/home/alvarocg/code/sanpigh/MRIsegmentation/ra...
3,TCGA_DU_7294_19890104,/home/alvarocg/code/sanpigh/MRIsegmentation/ra...
4,TCGA_DU_7294_19890104,/home/alvarocg/code/sanpigh/MRIsegmentation/ra...


time: 65.1 ms (started: 2021-07-06 15:42:50 +02:00)


In [111]:
len(dirs), len(images), len(masks)

(3929, 3929, 3929)

time: 5.06 ms (started: 2021-07-06 15:48:31 +02:00)


In [112]:
imagePath_df = pd.DataFrame({'directory': dirs, 'images': images, 'masks': masks})

time: 6.58 ms (started: 2021-07-06 15:48:35 +02:00)


In [113]:
imagePath_df

,directory,images,masks
0,TCGA_DU_7294_19890104,TCGA_DU_7294_19890104_30.tif,TCGA_DU_7294_19890104_30_mask.tif
1,TCGA_DU_7294_19890104,TCGA_DU_7294_19890104_3.tif,TCGA_DU_7294_19890104_3_mask.tif
2,TCGA_DU_7294_19890104,TCGA_DU_7294_19890104_26.tif,TCGA_DU_7294_19890104_26_mask.tif
3,TCGA_DU_7294_19890104,TCGA_DU_7294_19890104_19.tif,TCGA_DU_7294_19890104_19_mask.tif
4,TCGA_DU_7294_19890104,TCGA_DU_7294_19890104_9.tif,TCGA_DU_7294_19890104_9_mask.tif
...,...,...,...
3924,TCGA_DU_7309_19960831,TCGA_DU_7309_19960831_8.tif,TCGA_DU_7309_19960831_8_mask.tif
3925,TCGA_DU_7309_19960831,TCGA_DU_7309_19960831_37.tif,TCGA_DU_7309_19960831_37_mask.tif
3926,TCGA_DU_7309_19960831,TCGA_DU_7309_19960831_13.tif,TCGA_DU_7309_19960831_13_mask.tif
3927,TCGA_DU_7309_19960831,TCGA_DU_7309_19960831_11.tif,TCGA_DU_7309_19960831_11_mask.tif


time: 15.9 ms (started: 2021-07-06 15:48:41 +02:00)


In [115]:
!find ../raw_data/kaggle_3m -type f -print | grep mask | wc -l    # files with mask suffix
!find ../raw_data/kaggle_3m -type f -print | grep -v mask | wc -l # files without mask suffix

3929
3931
time: 717 ms (started: 2021-07-06 15:49:13 +02:00)


In [116]:
from os import walk # to navigate along directories
# from os.path import isfile, join
import tensorflow as tf

time: 1.79 ms (started: 2021-07-06 15:49:31 +02:00)


In [117]:
# Raw data
data_map = []
for sub_dir_path in glob.glob(DATA_PATH+"*"):
    if os.path.isdir(sub_dir_path):
        dirname = sub_dir_path.split("/")[-1]
        for filename in os.listdir(sub_dir_path):
            image_path = sub_dir_path + "/" + filename
            data_map.extend([dirname, image_path])
    else:
        print("This is not a dir:", sub_dir_path)
        
        
df = pd.DataFrame({"dirname" : data_map[::2],
                  "path" : data_map[1::2]})
df.head()

This is not a dir: /home/alvarocg/code/sanpigh/MRIsegmentation/raw_data/kaggle_3m/data.csv
This is not a dir: /home/alvarocg/code/sanpigh/MRIsegmentation/raw_data/kaggle_3m/README.md


,dirname,path
0,TCGA_DU_7294_19890104,/home/alvarocg/code/sanpigh/MRIsegmentation/ra...
1,TCGA_DU_7294_19890104,/home/alvarocg/code/sanpigh/MRIsegmentation/ra...
2,TCGA_DU_7294_19890104,/home/alvarocg/code/sanpigh/MRIsegmentation/ra...
3,TCGA_DU_7294_19890104,/home/alvarocg/code/sanpigh/MRIsegmentation/ra...
4,TCGA_DU_7294_19890104,/home/alvarocg/code/sanpigh/MRIsegmentation/ra...


time: 30.9 ms (started: 2021-07-06 15:49:36 +02:00)


In [11]:
!pip freeze | grep tensorflow

tensorflow==2.5.0
tensorflow-addons==0.13.0
tensorflow-estimator==2.5.0
tensorflow-io==0.19.0
tensorflow-io-gcs-filesystem==0.19.0


In [10]:
%load_ext autoreload
%autoreload 2

In [12]:
!pip install ipython-autotime

In [13]:
%load_ext autotime
%load_ext tensorboard

time: 7.88 ms (started: 2021-07-06 09:50:55 +02:00)


In [14]:
! pip install git+https://github.com/sanpigh/MRIsegmentation.git@cleanup --upgrade

  Cloning https://github.com/sanpigh/MRIsegmentation.git (to revision cleanup) to /tmp/pip-req-build-k5i4fskj
  Running command git clone -q https://github.com/sanpigh/MRIsegmentation.git /tmp/pip-req-build-k5i4fskj
  Running command git checkout -b cleanup --track origin/cleanup
  Switched to a new branch 'cleanup'
  Branch 'cleanup' set up to track remote branch 'cleanup' from 'origin'.
  Created wheel for MRIsegmentation: filename=MRIsegmentation-1.0-py3-none-any.whl size=9541 sha256=85181d606a17d172b3e4a24a3528f33fcf9e677d893380d4a2a691c3cf1085f1
  Stored in directory: /tmp/pip-ephem-wheel-cache-unyqpgpu/wheels/0e/d0/1f/1ee3d330eff737ab2b3d640bb9bd21c68fb46063228a5f6fa3
Successfully built MRIsegmentation
time: 4.01 s (started: 2021-07-06 09:51:10 +02:00)


In [20]:
EXPERIMENT_NAME = "[FR] [Bordeaux] [galleon] MRIsegmentation"
MLFLOW_URI = "https://mlflow.lewagon.co"
BUCKET_NAME = "galleon-mri"
BUCKET_DATA_PATH = "kaggle_3m"
GDRIVE_DATA_PATH = "/content/drive/MyDrive/MRIsegmentation/"

time: 841 µs (started: 2021-07-06 09:54:52 +02:00)


In [18]:
import cv2
import glob
import logging
import pandas as pd
import numpy as np
import os
import random

from sklearn.model_selection import train_test_split
from google.cloud import storage
import tensorflow as tf

from MRIsegmentation.params import GDRIVE_DATA_PATH, BUCKET_NAME, BUCKET_DATA_PATH

time: 831 µs (started: 2021-07-06 09:54:12 +02:00)


In [121]:
# from google.cloud import storage

# from google.colab import drive
# drive.mount('/content/drive')

time: 1.33 ms (started: 2021-07-06 15:53:04 +02:00)


In [132]:
from MRIsegmentation.data import get_data_from_drive, holdout

df = pd.read_csv(DATA_PATH + 'brain_df.csv')
df

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/MRIsegmentation//kaggle_3m/brain_df.csv'

time: 38.8 ms (started: 2021-07-06 16:12:18 +02:00)


In [123]:
def holdout(df, train_ratio=0.8, test_to_val_ratio=0.5, include_all=False):

    img_paths = df["image_path"].values
    msk_paths = df["mask_path"].values

    df_mask = df.copy()
    if include_all == False:
        df_mask = df[df["mask"] == 1]

    df_train, df_val = train_test_split(df_mask, train_size=train_ratio)
    df_test, df_val = train_test_split(df_val, test_size=test_to_val_ratio)

    ds_train = tf.data.Dataset.from_tensor_slices(
        (df_train["image_path"].values, df_train["mask_path"].values)
    )
    ds_val = tf.data.Dataset.from_tensor_slices(
        (df_val["image_path"].values, df_val["mask_path"].values)
    )
    ds_test = tf.data.Dataset.from_tensor_slices(
        (df_test["image_path"].values, df_test["mask_path"].values)
    )

    return ds_train, ds_val, ds_test

time: 1.81 ms (started: 2021-07-06 15:58:14 +02:00)


In [124]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# holdout
ds_train, ds_val, ds_test = holdout(df, include_all=True)

time: 327 ms (started: 2021-07-06 15:58:41 +02:00)


In [128]:
print(ds_train)
print(ds_val)
print(ds_test)

<TensorSliceDataset shapes: ((), ()), types: (tf.string, tf.string)>
<TensorSliceDataset shapes: ((), ()), types: (tf.string, tf.string)>
<TensorSliceDataset shapes: ((), ()), types: (tf.string, tf.string)>
time: 2.33 ms (started: 2021-07-06 16:00:55 +02:00)


## Create Trainer

In [129]:
import os
os.environ["TC_CPP_MIN_LOG_LEVEL"] = "2"

time: 1.7 ms (started: 2021-07-06 16:02:05 +02:00)


In [130]:
import tensorflow as tf
from MRIsegmentation.utils import tversky, tversky_loss, focal_tversky

t_ones = tf.constant([1.0, 1.0, 1.0])
t_null = tf.constant([0.0, 0.0, 0.0])

tversky(t_ones, t_null)

<tf.Tensor: shape=(), dtype=float32, numpy=4.761905e-08>

time: 467 ms (started: 2021-07-06 16:02:30 +02:00)


In [131]:
from MRIsegmentation.trainer import Trainer

# Define parameters to be evaluated

# hp_learning_rate = hp.Choice('learning_rate', values=[0.01, 0.05, 0.1])
# hp_learning_rate = hp.Choice('batch_size', values=[16, 32, 64])


LEARNING_RATE = 0.075
BATCH_SIZE = 16
EPOCHS = 2

params = dict(vgg19=dict(line_count=1_000, 
                         hyper_params=dict(learning_rate=LEARNING_RATE,
                                           batch_size=BATCH_SIZE,
                                           epochs=EPOCHS)),)

print(params.items())

# Run a trainer
trainer = Trainer()
mlflow_uri, result = trainer.train(params)

dict_items([('vgg19', {'line_count': 1000, 'hyper_params': {'learning_rate': 0.075, 'batch_size': 16, 'epochs': 2}})])


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/MRIsegmentation//kaggle_3m/brain_df.csv'

time: 4.46 s (started: 2021-07-06 16:05:42 +02:00)
